# RNN 
Didn't help with SimpleRNN: smooth the y_train, add hour to X.

Smooth the steam.

In [7]:
from os import listdir
import csv
from zipfile import ZipFile
import numpy as np
import pandas as pd
from scipy import stats  # mode

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential
from keras.layers import SimpleRNN
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import TimeDistributed
from keras.layers import Dense
from keras.losses import MeanSquaredError

import matplotlib.pyplot as plt
from matplotlib import colors
mycmap = colors.ListedColormap(['red','blue'])  # list color for label 0 then 1
np.set_printoptions(precision=2)

In [8]:
# Constants
EPOCHS=50
SITE = 'Eagle'
METER = 'steam'
PREDICTORS = ['hour','cloudCoverage', 'airTemperature', 'dewTemperature', 'precipDepth1HR', 'precipDepth6HR', 'seaLvlPressure', 'windDirection', 'windSpeed']
PREDICTORS = ['steam']
NUM_PREDICTORS=len(PREDICTORS)
print("PREDICTORS=",NUM_PREDICTORS,PREDICTORS)
PREDICTED_VARIABLE = 'steam'  
STEPS_HISTORY = 24
STEPS_FORWARD = 12 
STEPS_FUTURE =  12 
METER_FILE='steam.csv'
WEATHER_FILE='weather.csv'

PREDICTORS= 1 ['steam']


In [9]:
DATAPATH=''
try:
    # On Google Drive, set path to my drive / data directory.
    from google.colab import drive
    IN_COLAB = True
    PATH='/content/drive/'
    drive.mount(PATH)
    DATAPATH=PATH+'My Drive/data/'  # must end in "/"
except:
    # On home computer, set path to local data directory.
    IN_COLAB = False
    DATAPATH='data/'  # must end in "/"

ZIP_FILE='BuildingData.zip'
ZIP_PATH = DATAPATH+ZIP_FILE
MODEL_FILE='Model'  # will be used later to save models

In [10]:
def read_zip_to_panda(zip_filename,csv_filename):
    zip_handle = ZipFile(zip_filename)
    csv_handle = zip_handle.open(csv_filename)
    panda = pd.read_csv(csv_handle)
    return panda
def fix_date_type(panda):
    # Convert the given timestamp column to the pandas datetime data type.
    panda['timestamp'] = pd.to_datetime(panda['timestamp'], infer_datetime_format = True)
    indexed = panda.set_index(['timestamp'])
    return indexed


In [11]:
def load_weather_for_site(site):
    wet_df = read_zip_to_panda(ZIP_PATH,WEATHER_FILE)
    wet_df = fix_date_type(wet_df)
    site_df = wet_df.loc[wet_df['site_id'] == site]
    if False: # come back to this
        site_specific_weather.insert(0,'hour',0)
        L=len(site_specific_weather)
        for i in range(0,L):
            t=site_specific_weather.index[i]
            h=t.hour
            site_specific_weather.iat[i,0] = h
    return site_df

site_specific_weather = load_weather_for_site(SITE)
site_specific_weather.tail()

,site_id,airTemperature,cloudCoverage,dewTemperature,precipDepth1HR,precipDepth6HR,seaLvlPressure,windDirection,windSpeed
timestamp,,,,,,,,,
2017-12-31 18:00:00,Eagle,-11.1,0.0,-20.6,0.0,NaN,1026.2,330.0,2.6
2017-12-31 20:00:00,Eagle,-12.2,0.0,-21.1,0.0,NaN,1027.0,320.0,1.5
2017-12-31 21:00:00,Eagle,-12.8,0.0,-21.1,0.0,NaN,1027.2,310.0,2.6
2017-12-31 22:00:00,Eagle,-12.8,0.0,-20.6,0.0,NaN,1027.4,330.0,3.1
2017-12-31 23:00:00,Eagle,-12.8,0.0,-20.6,0.0,NaN,1027.4,320.0,4.6


In [15]:
def load_meter_for_site(site):
    all_df = read_zip_to_panda(ZIP_PATH,METER_FILE)
    all_df = fix_date_type(all_df)
    cols = [c for c in all_df.columns if c.startswith(site)]
    site_df = all_df[cols]
    #site_df = all_df.loc[all_df['site_id'] == site]
    return site_df

site_specific_meter = load_meter_for_site(SITE)
site_specific_meter.tail()

,Eagle_office_Lamont,Eagle_health_Athena,Eagle_assembly_Herbert,Eagle_public_Alvin,Eagle_education_Raul,Eagle_education_Roman,Eagle_office_Mandi,Eagle_education_Jewell,Eagle_office_Henriette,Eagle_health_Reba,...,Eagle_education_Petra,Eagle_lodging_Trina,Eagle_health_Reuben,Eagle_education_Teresa,Eagle_office_Norbert,Eagle_lodging_Casey,Eagle_office_Tia,Eagle_office_Remedios,Eagle_office_Patrice,Eagle_education_Shana
timestamp,,,,,,,,,,,,,,,,,,,,,
2017-12-31 19:00:00,65.0591,776.4879,627.9035,326.2933,1550.4797,2891.4935,51.0127,16.6959,0.0,1739.2564,...,184.0747,85.3863,736.2608,100.0444,585.9197,301.0408,0.0,213.9854,366.4862,146.6470
2017-12-31 20:00:00,64.5371,718.8517,653.3100,308.7546,1535.3834,2568.9456,50.9032,16.6959,0.0,1765.9299,...,175.8818,264.4709,708.4912,106.3892,613.8895,281.2951,0.0,212.2685,266.3693,146.3323
2017-12-31 21:00:00,68.7131,736.4628,675.0870,380.0056,1610.4618,2548.8111,50.5474,16.6959,0.0,1816.3536,...,193.7126,290.7334,733.7031,108.0552,582.2518,276.1160,0.0,226.0035,404.4867,145.3095
2017-12-31 22:00:00,68.2385,746.8694,613.3855,376.3517,1749.3991,2312.8049,50.5611,16.6959,0.0,1772.8722,...,186.8574,234.0364,732.2416,204.4267,573.8721,277.4107,0.0,224.4427,226.9072,145.3488
2017-12-31 23:00:00,66.7674,744.4678,640.6068,384.0249,1516.9403,2521.9345,50.3969,16.6959,0.0,1788.2187,...,183.9234,274.8555,749.0495,190.3427,623.9093,275.4685,0.0,219.6042,284.2734,144.8768


In [ ]:
# Building names
all_buildings = [x for x in site_specific_meter.columns if x.startswith(SITE)]

In [108]:
# Correlation is low when buildings have many NaN and 0 meter readings.
# We will ignore buildings that have >max bad meter readings.
def is_usable_column(df,column_name):
    MAX_BAD = 500 
    bad = df[column_name].isna().sum()
    return bad<=MAX_BAD

def prepare_for_learning(df):
    num_samples = len(df) - STEPS_FUTURE - STEPS_HISTORY
    X_shape = (num_samples,STEPS_HISTORY,NUM_PREDICTORS)
    X=np.zeros(X_shape)
    Y_shape = (num_samples,STEPS_FUTURE)
    y=np.zeros(Y_shape)
    predictor_series = df[PREDICTORS].values  # e.g. all weather values
    predicted_series = df[PREDICTED_VARIABLE].values  # e.g. all meter readings
    
    for sam in range (0,num_samples): # Loop over all 1000 samples
        # This is one array of weather for previous 24 time periods
        one_sample = predictor_series[sam:sam+STEPS_FORWARD]
        # Loop over all 24 time periods
        for time in range (0,STEPS_FORWARD): # In 1 sample, loop over 24 time periods
            one_period = one_sample[time]
            for feat in range (0,NUM_PREDICTORS): # In 1 time period, loop over 8 weather metrics
                X[sam,time,feat] = one_period[feat]
        for time in range (0,STEPS_FUTURE):  
            y[sam,time]=predicted_series[sam+STEPS_HISTORY+time]
    return X,y 


In [109]:
def make_RNN():
    rnn = Sequential([
        GRU(16,return_sequences=True, 
                  input_shape=(STEPS_HISTORY,NUM_PREDICTORS)), 
        GRU(16,return_sequences=True),
        GRU(16,return_sequences=False),
        Dense(STEPS_FUTURE)
    ])
    rnn.compile(optimizer='adam',loss=MeanSquaredError())
    return rnn

In [110]:
def window_smooth(oldarray):
    win_len=3
    df = pd.DataFrame(oldarray)
    newdf = df.rolling(win_len).mean()
    newarray = np.asarray(newdf)
    for i in range(0,win_len):
        newarray[i]=oldarray[i]
    return newarray


Pandas rolling() supports these window function from scipy:  
https://docs.scipy.org/doc/scipy/reference/signal.windows.html#module-scipy.signal.windows

In [111]:
cors = []
all_buildings = [x for x in site_specific_meter.columns if x.startswith(SITE)]
for BLDG in ['Eagle_lodging_Edgardo']:  ### all_buildings:
    print("Building",BLDG)
    # Get steam usage for one building.
    bldg_specific_steam = stm_df[[BLDG]]
    # Concatenate steam usage with weather.
    one_bldg_df = pd.concat([bldg_specific_steam,site_specific_weather],axis=1)
    # Drop the site, which is constant (we selected for one site).
    one_bldg_df = one_bldg_df.drop(['site_id'],axis=1)
    # The original steam table used column name = building name.
    # We are processing one building, so rename to the column 'steam'.
    one_bldg_df = one_bldg_df.rename(columns={BLDG : METER})
    # In order to filter bad buildings, count sum of NaN + zero.
    #### one_bldg_df = one_bldg_df.fillna(0)
    
    if is_usable_column(one_bldg_df,METER):
        pseudovalue = one_bldg_df[PREDICTED_VARIABLE].mean()
        one_bldg_df = one_bldg_df.fillna(pseudovalue)
        X,y = prepare_for_learning(one_bldg_df)
        split = len(X)//2   # year 1 vs year 2
        X_train = np.asarray(X[0:split])
        y_train = np.asarray(y[0:split])
        X_test = np.asarray(X[split:])
        y_test = np.asarray(y[split:])
        example=311
        model = make_RNN()
        print(model.summary())
        #print("Example X train:\n",X_train[example].astype(int))
        print("Example y train before smooth:\n",y_train[example].astype(int))
        y_train = window_smooth(y_train)
        print("Example y train after smooth:\n",y_train[example].astype(int))
        model.fit(X_train,y_train,epochs=EPOCHS)
        y_pred = model.predict(X_test)
        rmse = mean_squared_error(y_test,y_pred,squared=False)
        # Keep a table for reporting later.
        mean = one_bldg_df[METER].mean()
        cors.append([mean,rmse,rmse/mean,BLDG])
        print("mean,rmse,rmse/mean,bldg:",mean,rmse,rmse/mean,BLDG)
        for hr in range(0,24,2):
            print("Example prediction:\n",hr,y_pred[example+hr].astype(int))

print("History",STEPS_HISTORY,"Future",STEPS_FUTURE)
print("Column 1: Mean usage.")
print("Column 2: RMSE of LinearRegression(X=Weather, y=Usage).")
print("Column 3: RMSE/mean normalized to help understand RMSE.")
print("Column 4: Building.")
for cor in sorted(cors):
    print("%10.2f %10.2f %5.2f   %s"%(cor[0],cor[1],cor[2],cor[3]))    

Building Eagle_lodging_Edgardo
Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_27 (GRU)                 (None, 24, 16)            912       
_________________________________________________________________
gru_28 (GRU)                 (None, 24, 16)            1632      
_________________________________________________________________
gru_29 (GRU)                 (None, 16)                1632      
_________________________________________________________________
dense_10 (Dense)             (None, 12)                204       
Total params: 4,380
Trainable params: 4,380
Non-trainable params: 0
_________________________________________________________________
None
Example y train before smooth:
 [270 270 270 270 270 270 270 270 270 270 270 270]
Example y train after smooth:
 [270 270 270 270 270 270 270 270 270 270 270 270]
Epoch 1/50
274/274 [==============================